In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sacrebleu

In [3]:
# Load source and target text files
from datasets import Dataset

#source_file_path = '/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/merged_datasets/arabic-filtered_train.txt'
#target_file_path = '/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/merged_datasets/romanian-filtered_train.txt'
#
## Read the files into lists
#with open(source_file_path, 'r', encoding='utf-8') as f:
#    source_text = f.readlines()
#
#with open(target_file_path, 'r', encoding='utf-8') as f:
#    target_text = f.readlines()
#
## Strip newline characters
#source_text = [line.strip() for line in source_text]
#target_text = [line.strip() for line in target_text]
#
## Check if both files have the same number of lines
#assert len(source_text) == len(target_text), "The source and target files must have the same number of lines!"
#
## Create a dictionary with source and target
#data = {"source": source_text, "target": target_text}
## Create a Hugging Face Dataset from the dictionary
#dataset = Dataset.from_dict(data)


In [4]:
#dataset["source"][0]

'ما عليك فعله الأن هو أن تحصل على شهادتك'

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

source_language = "arb_Arab"
target_language = "ron_Latn"

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang=source_language, tgt_lang=target_language)

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
#from sklearn.model_selection import train_test_split
#
## Tokenize the dataset
#def preprocess_function(examples):
#    # Tokenize inputs and targets
#    inputs = examples["source"]
#    targets = examples["target"]
#
#    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=256,  return_tensors='pt')
#
#    # Use the target tokenizer for the targets
#    with tokenizer.as_target_tokenizer():
#        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
#
#    # Add labels to the model inputs (for training)
#    model_inputs["labels"] = labels["input_ids"]
#
#    # Return the tokenized inputs and labels
#    return model_inputs
#
#def tokenize_function(examples):
#    return tokenizer(
#        examples["source"],
#        examples["target"],
#        padding=True,        # Dynamic padding
#        truncation=True,     # Truncate longer sequences
#        max_length=256,      # Optional: specify maximum length if needed
#    )
#
## Split the dataset into train and validation sets (95% train, 5% validation)
#train_dataset, val_dataset = dataset.train_test_split(test_size=int(1000)).values()
#
## Apply preprocessing to both train and validation datasets
#tokenized_train_dataset = train_dataset.map(preprocess_function, num_proc=1, batched=True)  # Adjust num_proc based on the number of cores available
#tokenized_val_dataset = val_dataset.map(preprocess_function, num_proc=1, batched=True)

Map:   0%|          | 0/5437655 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [2]:
from datasets import load_from_disk

# Load the tokenized datasets from disk
tokenized_train_dataset = load_from_disk('/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/tokenized_train_dataset')
tokenized_val_dataset = load_from_disk('/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/tokenized_val_dataset')

Loading dataset from disk:   0%|          | 0/39 [00:00<?, ?it/s]

In [3]:

# Access encoder and decoder layers in the model
encoder_layers = model.model.encoder.layers
decoder_layers = model.model.decoder.layers

# Unfreeze the last 2 layers in the encoder
for i, layer in enumerate(encoder_layers):
    if i < len(encoder_layers) - 2:  # Freeze all layers except the last 2
        for param in layer.parameters():
            param.requires_grad = False
    else:
        for param in layer.parameters():
            param.requires_grad = True

# Unfreeze the last 2 layers in the decoder
for i, layer in enumerate(decoder_layers):
    if i < len(decoder_layers) - 2:  # Freeze all layers except the last 2
        for param in layer.parameters():
            param.requires_grad = False
    else:
        for param in layer.parameters():
            param.requires_grad = True

trainable_params = 0
frozen_params = 0

for param in model.parameters():
    if param.requires_grad:
        trainable_params += param.numel()  # numel() gives the number of elements in the tensor
    else:
        frozen_params += param.numel()

# Print the number of parameters in millions
print(f"Frozen parameters: {frozen_params / 1_000_000:.2f} million")
print(f"Trainable parameters: {trainable_params / 1_000_000:.2f} million")

Frozen parameters: 293.93 million
Trainable parameters: 321.14 million


In [6]:
!mkdir /content/saved_data

In [8]:
!pip install bert_score

In [4]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import evaluate
import numpy as np

import torch

metric = evaluate.load("bertscore")

def compute_bert(eval_pred):
    logits = eval_pred.label_ids
    predictions = eval_pred.predictions[0]
    # Decode the logits (model predictions) and labels (ground truth) into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(logits, skip_special_tokens=True)

    # Compute the BLEU score
    print(decoded_preds, [[label] for label in decoded_labels])
    result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels], lang="ro")
    return {"bert": np.mean(result['precision'])}

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

# Define TrainingArguments with saving the best model only
training_args = TrainingArguments(
    output_dir="/content/saved_data/",  # Output directory for the model and checkpoints
    eval_strategy="steps",  # Evaluate every specified number of steps
    eval_steps=2000,  # Evaluate every 5000 steps
    learning_rate=5e-5,
    per_device_train_batch_size=16, #20 batch size
    per_device_eval_batch_size = 8,
    eval_accumulation_steps=32,
    num_train_epochs=3,
    save_total_limit=1,  # Keep only the best model (1 checkpoint)
    save_strategy = "best",
    load_best_model_at_end=True,  # Reload the best model based on evaluation metric
    metric_for_best_model="eval_bert",  # Use BLEU as the metric for the best model
    greater_is_better=True,  # For BLEU, a higher score is better
    remove_unused_columns=True,
    report_to="none",
)

# Define the EarlyStoppingCallback (optional, for stopping early)
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations to wait for improvement
    early_stopping_threshold=0.0,  # Minimum change to qualify as an improvement
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_bert,  # Use custom metrics (BLEU)
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    callbacks=[early_stopping_callback],  # Optional: Early stopping callback
)

# Train the model
trainer.train()

<ipython-input-4-8e587ac18ad2>:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Bert
2000,0.142400,0.127821,0.836718
4000,0.137300,0.125785,0.837480
6000,0.136100,0.124747,0.839622
8000,0.136800,0.123885,0.839257
10000,0.136600,0.123433,0.840102
12000,0.136700,0.122470,0.841559
14000,0.133300,0.122097,0.840701
16000,0.132000,0.121586,0.840397
18000,0.131500,0.121192,0.841161


['Flirtbox este un site de dating gratuit în caracterul din Irlanda, în timp ce Parship este un site de dating online în Franța.', 'Acestastă tendință este dată în creșterea preculoasă a prețurilor anulată de trecut de criza monetară, cu scă scăăbușire a prelei, precum și prin interul alimentarit de Kremlin asupra majorității produselor alimentare occidentale, ca răspunsie ale adresațiunile europene și americane din urmaazătele crizei dinrainene.', 'Noua Zeelandanda - Nou una dintre cele mai active ț din lume, o o greor de are la competi sem la concitii sportive internationale.', 'Nu pare că ucigaşul nostru a folosit cheurile pentru la înc înctul ăsta. să intre.', 'Dar s-a întâmplat, amoi amje clubul.', 'O hrăni şi voiăp caiii. Ing şiăuntru şi adu-feauabinte.', 'E fost e singura mea trimaă, Grant.', 'Pentru văd dacă sunt înficat să a intrami accesa corpulul de la', 'Asta pentru AD nu are prea mulţi prieteni.', 'E e ceva altceva aici afară de furie?', 'A mult de împrumut bani, la tine, 

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


['Flirtbox este un site de dating gratuit în caracterul în Irlanda, în timp ce Parship este un site de dating online în Franța.', 'Acestastă situaință este dată dat creșterea preculoasă a prețurilor anulată de trecut de criza monetară, cu scă scăăbușire a prel, precum și prin interul alimentarit de Kremlin asupra majorității mărelor alimentare occidentale, ca răspunsie ale adresațiunile UEene și americane din urmaazătu crizei dinrainene.', 'Noua Zeelandanda - Nou una dintre cele mai active ț din lume, o o mareor de are la compet sem la competitii sportive internationale.', 'Nu pare că ucigaşul nostru a folosit cheurile pentru la înccătul ăsta. să intre.', 'Dar s-a întâmplat, amoi amje clubul.', 'O hrăni şi voiăuga caiii. Intrgi şiăuntru şi adu-fea caldbinte.', 'E fost e singura mea trimaă, Grant.', 'Să văd dacă sunt înficat să a intrami accesa corpulul de la', 'Asta pentru AD nu are prea mulţi prieteni.', 'E e ceva altceva aici afară de furie?', 'A mult de împrumu bani, la tine, Bob, ş

TrainOutput(global_step=18000, training_loss=0.2169882024129232, metrics={'train_runtime': 15694.4414, 'train_samples_per_second': 1039.41, 'train_steps_per_second': 64.963, 'total_flos': 1.56031531352064e+17, 'train_loss': 0.2169882024129232, 'epoch': 0.052963919800855665})

In [ ]:
!cp -r /content/saved_data/*  /content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/

In [ ]:
from google.colab import runtime
runtime.unassign()

In [12]:
# Save the tokenized datasets to disk
tokenized_train_dataset.save_to_disk('/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/tokenized_train_dataset')
tokenized_val_dataset.save_to_disk('/content/drive/MyDrive/disertatie/FilteredDatasets/Eliminated_dialects/NLLB/tokenized_val_dataset')

Saving the dataset (0/39 shards):   0%|          | 0/5437655 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]